# Motifs Probabilísticos

Os **motifs** probabilísticos em Python referem-se a padrões ou sequências de eventos que ocorrem com uma certa probabilidade num conjunto de dados. 

Em bioinformática, por exemplo, os **motifs** podem representar padrões de nucleótidos em sequências de ADN, podendo também ser utilizados para encontrar regiões conservadas em sequências de ADN ou RNA.

As funções, **PWM** e **PSSM**, são ferramentas essenciais na análise de dados genómicos, na identificação de elementos regulatórios importantes nas sequências do ADN e na identificação de **motifs** conservados em regiões regulatórias do ADN. 

> ### Função PWM

A **função PWM** (Position Weight Matrix) é uma técnica amplamente utilizada na bioinformática para modelar padrões de consenso em sequências biológicas, como ADN ou proteínas. 

Consiste numa matriz que representa a frequência relativa de cada base em cada posição ao longo de um conjunto de sequências alinhadas.

Em primeiro lugar, procedeu-se à instalação da biblioteca tabulate que permite a utilização de um conjunto de ferramentas de forma a ter um output mais organizado.

In [6]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.



A função **pwm** tem como objetivo calcular a matriz PWM (Matriz de Peso e Posição) para um conjunto de sequências fornecidas. Ela recebe uma lista de strings que representam as sequências (seqs) e um valor opcional chamado pseudo (pseudo), que por padrão é igual a 0.

A função realiza algumas verificações iniciais, garantindo que as sequências fornecidas são válidas. Em caso de sequências inválidas, a função lança um AssertionError. Além disso, ela verifica se cada elemento da lista de sequências é uma string.

A função utiliza um alfabeto padrão de DNA ('ACGT') e, para cada posição em todas as sequências, calcula a frequência relativa de cada base (A, C, G, T) naquela posição. O cálculo considera um termo de pseudo contabilizado para evitar divisões por zero. O resultado é uma lista de dicionários, onde cada dicionário representa uma posição na matriz PWM. As chaves dos dicionários são as bases do DNA ('A', 'C', 'G', 'T'), e os valores são as frequências relativas calculadas.

Por fim, a função retorna a matriz PWM como uma lista de dicionários.

In [70]:
def pwm(seqs: list[str], pseudo: float = 0) -> list[dict[str, float]]:
  
  """
  Calcula a matriz PWM (Matriz de Peso e Posição) para as sequências fornecidas

  Parâmetros
  -------------
  seqs : list[str]
      Recebe uma lista de strings que representam as sequências

  pseudo : float
      Recebe um valor opcional, pseudo, que em caso de omissão é = 0


  Retorna
  -------------
  pwm_matrix : list[dict[str, float]]
      Retorna uma *lista de dicionários*, onde cada *dicionário* terá uma chave no formato de *string*, e um valor no formato de *float*

  Levanta
  -------------
  AssertError
      Caso a lista de sequências contenha sequências inválida
  

  """
  
  from scripts.auxiliares import validar_dna

  for seq in seqs:
    assert validar_dna(seq), ("Sequência inválida")
    
    
  alfabeto = 'ACGT'

  for seq in seqs:
    for idx, b in enumerate(seq):
      assert b in alfabeto, f'Caracter {b} na posição {idx} da sequência {seq} inválido!'

  pwm_matrix = [{b: (pos.count(b) + pseudo) / (len(seqs) + len(alfabeto) * pseudo)
    for b in alfabeto}
      for pos in zip(*seqs)]

  return pwm_matrix

**Exemplo**:

In [73]:
seqs = ['TGACTATACGTATGGTAGAT', 'ATCGTATACGTAGGTAGAC', 'TAGCTAGTCGTATGGTAGAT']
pseudo = 0

# Associação do resultado da função a uma variável de forma a ser mais fácil imprimir o resultado
resultado = pwm(seqs, pseudo)

# Impressão da matriz formatada com o módulo tabulate
from tabulate import tabulate
headers = ["Base"] + [f"Posição {i+1}" for i in range(len(seqs[0]))]
table = [[base] + [round(resultado[i][base], 3) for i in range(len(resultado))] for base in "ACGT"]
print(tabulate(table, headers))


Base      Posição 1    Posição 2    Posição 3    Posição 4    Posição 5    Posição 6    Posição 7    Posição 8    Posição 9    Posição 10    Posição 11    Posição 12    Posição 13    Posição 14    Posição 15    Posição 16    Posição 17    Posição 18    Posição 19
------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------
A             0.333        0.333        0.333        0                0            1        0            0.667            0             0             0             1         0                 0         0             0.333         0.667         0.333         0.667
C             0            0            0.333        0.667            0            0        0            0                1             0             0             0         0                 0         0     

**Testes de unidade**:

In [42]:
import unittest

class TestPWM(unittest.TestCase):
    def teste_formula(self):
        self.assertEqual(pwm(["AA", "AA"]), [{'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}, {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}])

    def teste_seq_invalid(self):
        with self.assertRaises(AssertionError):
            pwm([1, "ATC"])
    
    def test_seq_vazio(self):
        with self.assertRaises(AssertionError):
            pwm(["", "ATC"])

suite = unittest.TestLoader().loadTestsFromTestCase(TestPWM)
unittest.TextTestRunner( verbosity=3 ).run( suite )

test_seq_vazio (__main__.TestPWM) ... ok
teste_formula (__main__.TestPWM) ... ok
teste_seq_invalid (__main__.TestPWM) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.015s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

Foi desenvolvida uma função denominada **prob_seq** que calcula a probabilidade de uma sequência utilizando como base a função PWM.

A função calcula a probabilidade de uma determinada sequência de ADN com base na matriz PWM (Matriz de Peso e Posição). Em primeiro lugar,inicializa a variável probabilidade com o valor 1.0. De seguida, itera sobre cada posição e base na sequência fornecida. Para cada base, a função verifica se a base está presente no dicionário correspondente à posição na matriz PWM (resultado). Se estiver, multiplica a probabilidade acumulada pela probabilidade da base naquela posição. Se a base não estiver presente, atribui uma probabilidade mínima de 0.01 à base ausente.

Ao percorrer toda a sequência, a função retorna a probabilidade final da sequência com base na matriz PWM fornecida.

In [43]:
def prob_seq(sequence, resultado):
    """
    Calcula a probabilidade de uma sequência utilizando como base a função PWM

    
    Parâmetros
    -------------
    sequence : str 
        sequência de DNA 
     
    resultado : list[dict[str, float]] 
        Variável que contém o resultado da função PWM, onde cada dicionário representa as probabilidades para cada base numa posição

        
    Retorna
    -------------
    probabilidade : float 
        A probabilidade da sequência com base na PWM

        
    Levanta
    -------------
    AssertError
      Caso a lista de sequências contenha sequências inválida
    
    """
    
    from scripts.auxiliares import validar_dna
    
    assert validar_dna(seq), ("Sequência inválida")
    
    probabilidade = 1.0

    for position, base in enumerate(sequence):
        if base in resultado[position]:
            probabilidade *= resultado[position][base]
        else:
            probabilidade *= 0.01  # Atribui uma probabilidade mínima de 0.01 para bases ausentes.

    return probabilidade

**Exemplo**:

In [6]:
seq = 'ATA'
probabilidade = prob_seq(seq, resultado)

print(f"A probabilidade da sequência '{seq}' é de: \n{probabilidade}")

A probabilidade da sequência 'ATA' é de: 
0.037037037037037035


**Testes de unidade**:

Seguidamente, foi criada a função **seq_provavel** que tem como objetivo calcular a sequência mais provável dentro de uma sequência de ADN, com base na matriz PWM (Matriz de Peso e Posição).

A função inicia um dicionário vazio chamado "dicionario". De seguida, utiliza expressões regulares para dividir a sequência em subconjuntos de quatro bases, deslizando uma janela de tamanho 4 pela sequência. Para cada subconjunto, a função calcula a probabilidade usando a função prob_seq e armazena a probabilidade associada ao subconjunto no dicionário.

No final do processo, a função retorna a sequência de quatro bases que possui a maior probabilidade, ou seja, aquela associada ao maior valor no dicionário de probabilidades. Isso é realizado utilizando a função max com a chave sendo a probabilidade associada a cada subconjunto. Portanto, a função retorna a subsequência de quatro bases que é considerada mais provável com base na matriz PWM fornecida.

In [45]:
def seq_provavel(seq, resultado):
  
  '''
  Calcula qual a Sequência mais provável

  
  Parâmetros
  -------------
  seq : str
    A sequência de DNA
  
  resultado : list[dict[str, float]] 
    Variável que contém o resultado da função PWM, onde cada dicionário representa as probabilidades para cada base em uma posição
  
  
  Retorna
  -------------
  str 
    A Sequência mais provável dentro da Sequência dada

    
  Levanta
  -------------
  AssertError
      Caso a lista de sequências contenha sequências inválida
  
  '''
  import re

  from scripts.auxiliares import validar_dna
    
  assert validar_dna(seq), ("Sequência inválida")
  
  dicionario = {}
  for subset in re.findall('(?=(....))', seq):
    dicionario[subset] = (prob_seq(subset, resultado))
  return max(dicionario, key=dicionario.get)

**Exemplo**:

In [46]:
seq = "CATTGT"
mais_provavel = seq_provavel(seq, resultado)

print(f'A Sequência mais provável, dentro da Sequência {seq}, é: {mais_provavel}.')

A Sequência mais provável, dentro da Sequência CATTGT, é: CATT.


In [68]:
seq = "AAT"
resultado = pwm(["AATCCTT", "TAATAGCCT", "AATTGCGCT"])

In [69]:
seq_provavel(seq, resultado)

ValueError: max() arg is an empty sequence

In [22]:
import unittest

class Testseq_provavel(unittest.TestCase):

    def teste_6(self):
        self.assertEqual(seq_provavel(["AA", "AA"]), [{'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}, {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}])

    def teste_seq_invalid(self):
        with self.assertRaises(AssertionError):
            seq_provavel(["AAT", ])
    
    def test_seq_vazio(self):
        with self.assertRaises(AssertionError):
            seq_provavel(["", "ATC"])

suite = unittest.TestLoader().loadTestsFromTestCase(Testseq_provavel)
unittest.TextTestRunner( verbosity=3 ).run( suite )


test_seq_vazio (__main__.Testseq_provavel) ... ERROR
teste_6 (__main__.Testseq_provavel) ... ERROR
teste_seq_invalid (__main__.Testseq_provavel) ... ERROR

ERROR: test_seq_vazio (__main__.Testseq_provavel)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\olive\AppData\Local\Temp\ipykernel_16600\926319873.py", line 14, in test_seq_vazio
    seq_provavel(["", "ATC"])
NameError: name 'seq_provavel' is not defined

ERROR: teste_6 (__main__.Testseq_provavel)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\olive\AppData\Local\Temp\ipykernel_16600\926319873.py", line 6, in teste_6
    self.assertEqual(seq_provavel(["AA", "AA"]), [{'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}, {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}])
NameError: name 'seq_provavel' is not defined

ERROR: teste_seq_invalid (__main__.Testseq_provavel)
-----------------------------------

<unittest.runner.TextTestResult run=3 errors=3 failures=0>

**Testes de unidade**:

>### Função PSSM

A **função PSSM** (Position-Specific Scoring Matrix) é uma extensão da PWM. 

Enquanto a PWM representa a frequência relativa das bases, a PSSM atribui uma pontuação para cada base em cada posição, levando em consideração a probabilidade logarítmica da ocorrência de uma base em relação a uma distribuição de probabilidade de fundo.

In [14]:
def pssm(seqs, pseudo = 1):

  """
  Calcula a Matriz de Pontuação de Posição Específica (PSSM) para um conjunto de sequências de ADN

  Parâmetros
  -------------
  seqs : list[str]
    Lista de sequências de ADN

  pseudo : float 
    Valor de pseudocount a ser adicionado para evitar problemas com probabilidades zero.


  Retorna
  -------------
  lista : list[dict[str, float]]
    Uma lista de dicionários que representa a PSSM
  
    
  Levanta
  -------------
  AssertError
      Caso a lista de sequências contenha sequências inválida
  
  """
  import math

  from scripts.auxiliares import validar_dna

  for seq in seqs:
    assert validar_dna(seq), ("Sequência inválida")

  bases = 'ATCG'
  lista = []

  for pos in list(zip(*seqs)):
    dicionario = {}
    for b in bases:
      # Fórmula da PSSM: log2((contagem da base + pseudocount) / (total de sequências + total de bases * pseudocount)) / 0.25
      dicionario[b] = math.log2((pos.count(b) + pseudo) / (len(seqs) + len(bases)*pseudo)) / 0.25

    lista.append(dicionario)
  return lista

Base      Posição 1    Posição 2    Posição 3    Posição 4    Posição 5    Posição 6    Posição 7    Posição 8    Posição 9    Posição 10    Posição 11    Posição 12    Posição 13    Posição 14    Posição 15    Posição 16    Posição 17    Posição 18    Posição 19
------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------
A            -7.229       -7.229       -7.229      -11.229      -11.229       -3.229      -11.229       -4.89       -11.229       -11.229       -11.229        -3.229       -11.229       -11.229       -11.229        -7.229        -4.89         -7.229        -4.89
C           -11.229      -11.229       -7.229       -4.89       -11.229      -11.229      -11.229      -11.229       -3.229       -11.229       -11.229       -11.229       -11.229       -11.229       -11.229  

**Exemplo**:

In [15]:
seqs = ['TGACTATACGTATGGTAGAT', 'ATCGTATACGTAGGTAGAC', 'TAGCTAGTCGTATGGTAGAT']
matriz_pssm = pssm(seqs, pseudo = 1)

# O Table e o Headers já foram definidos para a função PWM

headers = ["Base"] + [f"Posição {i+1}" for i in range(len(seqs[0]))]
table = [[base] + [round(matriz_pssm[i][base], 3) for i in range(len(matriz_pssm))] for base in "ACGT"]
print(tabulate(table, headers))

Base      Posição 1    Posição 2    Posição 3    Posição 4    Posição 5    Posição 6    Posição 7    Posição 8    Posição 9    Posição 10    Posição 11    Posição 12    Posição 13    Posição 14    Posição 15    Posição 16    Posição 17    Posição 18    Posição 19
------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  -----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------
A            -7.229       -7.229       -7.229      -11.229      -11.229       -3.229      -11.229       -4.89       -11.229       -11.229       -11.229        -3.229       -11.229       -11.229       -11.229        -7.229        -4.89         -7.229        -4.89
C           -11.229      -11.229       -7.229       -4.89       -11.229      -11.229      -11.229      -11.229       -3.229       -11.229       -11.229       -11.229       -11.229       -11.229       -11.229  

In [53]:
import unittest

class TestPSSMFunctions(unittest.TestCase):

    def setUp(self):
        # Configuração inicial comum aos testes
        self.seqs = ['TGACTATACGTATGGTAGAT', 'ATCGTATACGTAGGTAGAC', 'TAGCTAGTCGTATGGTAGAT']
        self.pseudo = 1
        self.resultado_pssm = pssm(self.seqs, self.pseudo)

    def test_pssm_shape(self):
        # Verifica se o número de posições e bases está correto
        self.assertEqual(len(self.resultado_pssm), len(self.seqs[0]))
        self.assertEqual(set(self.resultado_pssm[0].keys()), set('ATCG'))

    def test_pssm_values(self):
        # Verifica se os valores da PSSM estão corretos
        expected_values = [
            {'........
            
        ]
        for i, expected in enumerate(expected_values):
            for base, value in expected.items():
                self.assertAlmostEqual(self.resultado_pssm[i][base], value, places=3)

if __name__ == '__main__':
    # Remova a opção -f/--failfast ao chamar unittest.main()
    unittest.main(argv=[''], exit=False)


FFFEFFF...F.F.
ERROR: test_pssm_with_pseudo_0 (__main__.TestPSSM.test_pssm_with_pseudo_0)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\armin\AppData\Local\Temp\ipykernel_19352\837469118.py", line 57, in test_pssm_with_pseudo_0
    result = pssm(seqs, pseudo=0)
             ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\armin\AppData\Local\Temp\ipykernel_19352\776625500.py", line 36, in pssm
    dicionario[b] = math.log2((pos.count(b) + pseudo) / (len(seqs) + len(bases)*pseudo)) / 0.25
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: math domain error

FAIL: test_prob_seq (__main__.TestMyFunctions.test_prob_seq)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "C:\Users\armin\AppData\Local\Temp\ipykernel_19352\1639364311.py", line 19, in test_prob_seq
    self.assertAlmostEqual(probabilidade, 0.123, place

**Testes de unidade**: